In [19]:
import pandas as pd
import re

In [20]:
df = pd.read_excel('pesquisa-viajou.xlsx')

Alterando nome das colunas

In [21]:
colunas = ['id','hora_inicio','hora_conclusao','email','nome','idade','genero','municipio_residencia','visita_pontos_turisticos','participa_eventos','frequencia_eventos','tipos_eventos_preferidos','participa_excursões','frequencia_tours_virtuais','frequencia_apps_turismo','usou_apps_turismo','confianca_avaliacoes','interesse_acessibilidade','pagaria_por_tour_virtual','descoberta_eventos_atracoes','motivacao_uso_app_turismo','probabilidade_recomendar']

df.columns = colunas
df.drop(['id', 'hora_inicio', 'hora_conclusao', 'email', 'nome'], axis=1, inplace=True)


df['municipio_residencia']

0      São Paulo (capital)
1      São Paulo (capital)
2      São Paulo (capital)
3      São Paulo (capital)
4                   Osasco
              ...         
126               Campinas
127    São Paulo (capital)
128              Guarulhos
129    São Paulo (capital)
130    São Paulo (capital)
Name: municipio_residencia, Length: 131, dtype: object

Limpando dados desnecessários

Arrays permitidos

In [22]:
municipios_permitidos = ['São Paulo', 'Osasco', 'Guarulhos', 'Atibaia', 'Carapicuíba ', 'Barueri', 'Caieiras', 'São Bernardo do Campo', 'Santana de Parnaíba.']

tipos_eventos_permitidos = [
    'Musicais',
    'Teatrais',
    'Culturais',
    'Gastronômicos',
    'Esportivos',
    'Eletrônicos'
]

descoberta_eventos_atracoes_permitidos = [
    'Redes Sociais',
    'Sites de Turismo',
    'Blogs e Sites de Viagem',
    'Aplicativos de Turismo',
    'Recomendações de Amigos e Família',
    'Agências de Viagem e Operadoras de Turismo'
]

motivacao_uso_app_turismo_permitidos = [
    'Encontrar Novas Experiências',
    'Obter Informações Detalhadas',
    'Reservar Ingressos e Excursões',
    'Receber Recomendações Personalizadas',
    'Acompanhar Eventos e Ofertas Especiais',
    'Nenhuma das opções'
]

In [23]:
df = df.map(lambda x: re.sub(r"\s*\([^)]*\)", "", x) if isinstance(x, str) else x)

df['municipio_residencia'] = df['municipio_residencia'].apply(lambda x: x if x in municipios_permitidos else None)

df['descoberta_eventos_atracoes'] = df['descoberta_eventos_atracoes'].apply(lambda x: [item for item in x.split(';') if item.strip() in descoberta_eventos_atracoes_permitidos] if isinstance(x, str) else x)
df['descoberta_eventos_atracoes'] = df['descoberta_eventos_atracoes'].apply(lambda x: [item.strip() for item in x] if isinstance(x, list) else x)
df['descoberta_eventos_atracoes'] = df['descoberta_eventos_atracoes'].apply(lambda x: x if isinstance(x, list) and len(x) > 0 else None)
df['descoberta_eventos_atracoes'] = df['descoberta_eventos_atracoes'].apply(lambda x: sorted(x) if isinstance(x, list) else x)

df['tipos_eventos_preferidos'] = df['tipos_eventos_preferidos'].apply(lambda x: [item for item in x.split(';') if item.strip() in tipos_eventos_permitidos] if isinstance(x, str) else x)
df['tipos_eventos_preferidos'] = df['tipos_eventos_preferidos'].apply(lambda x: [item.strip() for item in x] if isinstance(x, list) else x)
df['tipos_eventos_preferidos'] = df['tipos_eventos_preferidos'].apply(lambda x: x if isinstance(x, list) and len(x) > 0 else None)
df['tipos_eventos_preferidos'] = df['tipos_eventos_preferidos'].apply(lambda x: sorted(x) if isinstance(x, list) else x)

df['motivacao_uso_app_turismo'] = df['motivacao_uso_app_turismo'].apply(
    lambda x: x if x in motivacao_uso_app_turismo_permitidos else None
)

In [24]:
df = df.dropna()

df.to_excel('viajou-tratado.xlsx', index=False)

,idade,genero,municipio_residencia,visita_pontos_turisticos,participa_eventos,frequencia_eventos,tipos_eventos_preferidos,participa_excursões,frequencia_tours_virtuais,frequencia_apps_turismo,usou_apps_turismo,confianca_avaliacoes,interesse_acessibilidade,pagaria_por_tour_virtual,descoberta_eventos_atracoes,motivacao_uso_app_turismo,probabilidade_recomendar
0,Menor de 18 anos,Feminino,São Paulo,Sim,Sim,Ocasionalmente,[Teatrais],Não,Nunca,Ocasionalmente,Sim,3,"Sim, mas não é essencial","Não, prefiro gastar em outra coisa",[Redes Sociais],Encontrar Novas Experiências,3
1,Menor de 18 anos,Masculino,São Paulo,Não,Sim,Ocasionalmente,"[Eletrônicos, Gastronômicos, Musicais]",Não,Nunca,Raramente,Sim,5,"Sim, mas não é essencial","Sim, se o preço for acessível","[Recomendações de Amigos e Família, Redes Soci...",Obter Informações Detalhadas,4
2,Menor de 18 anos,Feminino,São Paulo,Não,Não,Raramente,[Gastronômicos],Não,Nunca,Raramente,Sim,4,"Sim, mas não é essencial","Sim, se o preço for acessível","[Recomendações de Amigos e Família, Redes Soci...",Encontrar Novas Experiências,4
3,Mais de 55 anos,Feminino,São Paulo,Sim,Não,Ocasionalmente,"[Culturais, Musicais]",Não,Raramente,Ocasionalmente,Sim,3,"Sim, é muito importante pra mim.","Sim, se o preço for acessível",[Agências de Viagem e Operadoras de Turismo],Obter Informações Detalhadas,3
4,Menor de 18 anos,Outro,Osasco,Sim,Sim,Ocasionalmente,"[Culturais, Musicais, Teatrais]",Sim,Raramente,Raramente,Não,4,"Sim, é muito importante pra mim.","Talvez, depende da situação","[Recomendações de Amigos e Família, Redes Soci...",Encontrar Novas Experiências,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Mais de 55 anos,Masculino,São Paulo,Não,Não,Raramente,"[Esportivos, Gastronômicos]",Não,Nunca,Nunca,Não,4,"Sim, mas não é essencial","Não, prefiro gastar em outra coisa",[Recomendações de Amigos e Família],Receber Recomendações Personalizadas,3
123,35-44 anos,Masculino,Osasco,Não,Sim,Mensalmente,"[Esportivos, Gastronômicos]",Sim,Raramente,Sempre que saio.,Sim,4,"Sim, é muito importante pra mim.","Sim, se o preço for acessível",[Aplicativos de Turismo],Reservar Ingressos e Excursões,1
125,25-34 anos,Feminino,São Paulo,Sim,Sim,Ocasionalmente,"[Eletrônicos, Esportivos, Gastronômicos, Music...",Não,Ocasionalmente,Ocasionalmente,Sim,4,"Sim, mas não é essencial","Não, prefiro gastar em outra coisa","[Recomendações de Amigos e Família, Redes Soci...",Reservar Ingressos e Excursões,2
129,Mais de 55 anos,Feminino,São Paulo,Sim,Sim,Ocasionalmente,[Culturais],Não,Ocasionalmente,Frequentemente,Sim,5,"Sim, mas não é essencial","Sim, se o preço for acessível","[Agências de Viagem e Operadoras de Turismo, S...",Obter Informações Detalhadas,4


: 